[Node 37: TDD Beispiel](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node37.html)

Navigation:

 **Next:** [Mit PDB debuggen](node38.ipynb) **Up:** [Entwicklung mit Tests](node36.ipynb) **Previous:** [Entwicklung mit Tests](node36.ipynb)

##  Beispiel für TDD
Als Beispiel zum TDD soll ein sehr einfacher Taschenrechner dienen, der in der   <font color=#0000e6> ``add``</font>-Methode die Addition zweier Zahlen ausführt und das Ergebnis zurückgibt. Zunächst wird ein leeres Projekt angelegt:

```bash
mkdir mytest
cd mytest
mkdir app
mkdir test
touch app/__init__.py
touch test/__init__.py
```

 Im Verzeichnis   <font color=#0000e6> ``test``</font>  wird die Datei   <font color=#0000e6> ``test_rechner.py``</font>  mit folgendem Inhalt erzeugt:  

```python
import unittest
 
class TddBeispiel(unittest.TestCase):
 
    def test_rechner_add_method_gibt_richtiges_ergebnis(self):
        rech = Rechner()
        res = rech.add(2,2)
        self.assertEqual(4, res)

if __name__ == '__main__':
    unittest.main()
```


Ein Test hat folgende Struktur:  
* Import des Moduls   <font color=#0000e6> ``unittest``</font> . 
* Anlegen einer von   <font color=#0000e6> ``unittest.TestCase``</font>  abgeleiteten Klasse, die alle Tests beinhaltet. 
* Alle Methoden dieser Klasse implementieren verschiedene Tests und müssen mit   <font color=#0000e6> ``test_``</font>  beginnen. 
* Die letzten beiden Zeilen ermöglichen es, den Test mit dem Standard   <font color=#0000e6> ``unittest``</font>  Modul und dem Befehl   <font color=#0000e6> ``python test_rechner.py``</font>   auszuführen.  

Das Programm   <font color=#0000e6> ``nosetests3``</font>  ermöglicht das automatische Ausführen aller existierenden Tests:

```bash
> nosetests3
```
<pre>
E
======================================================================
ERROR: test_rechner_add_method_gibt_richtiges_ergebnis (test_rechner.TddBeispiel)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/gduckeck/python/test/mytest/test_rechner.py", line 6, in test_rechner_add_method_gibt_richtiges_ergebnis
    rech = Rechner()
NameError: name 'Rechner' is not defined
----------------------------------------------------------------------
Ran 1 test in 0.004s
FAILED (errors=1)
</pre>

 <font color=#0000ff> **Fehler war provoziert**</font>  ... es fehlt noch die tatsächliche Implementierung von   <font color=#0000e6> ``Rechner``</font> . Es wird also die Datei   <font color=#0000e6> ``app/rechner.py``</font>  erzeugt:
```python
class Rechner(object):
 
    def add(self, x, y):
        pass
```
und der Test erweitert:  
```python
import unittest
from app.rechner import Rechner # neu

class TddBeispiel(unittest.TestCase):
 
    def test_rechner_add_method_gibt_richtiges_ergebnis(self):
        rech = Rechner()
        res = rech.add(2,2)
        self.assertEqual(4, res)

if __name__ == '__main__':
    unittest.main()
```

Ein erneutes Ausführen der Tests ergibt:
<pre>
> nosetests3
F
======================================================================
FAIL: test_rechner_add_method_gibt_richtiges_ergebnis (test_rechner.TddBeispiel)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/gduckeck/python/test/mytest/test_rechner.py", line 9, in test_rechner_add_method_gibt_richtiges_ergebnis
    self.assertEqual(4, res)
AssertionError: 4 != None
----------------------------------------------------------------------
Ran 1 test in 0.004s
FAILED (failures=1)
</pre>

 Der Test zeigt an, daß die Methode    <font color=#0000e6> ``add``</font>   noch kein richtiges  Ergebnis liefert. Dies kann folgendermaßen korrigiert werden:
```python
class Rechner(object):
 
    def add(self, x, y):
        return x+y # neu
```

<pre>
> nosetests3 
.
----------------------------------------------------------------------
Ran 1 test in 0.019s
OK
</pre>

Der Test funktioniert jetzt, allerdings wird nur der Fall getestet, der tatsächlich zunächst interessiert - was passiert allerdings, falls andere Typen als Zahlen verwendet werden, da Python das Addieren von z.B. Strings oder Listen mit der gleichen Syntax erlaubt? Um diese Fälle zu testen, wird der Test erweitert:  
```python
import unittest
from app.rechner import Rechner

class TddBeispiel(unittest.TestCase):
  
    def setUp(self): # neu
        self.rech = Rechner()

    def test_rechner_add_method_gibt_richtiges_ergebnis(self):
        res = self.rech.add(2,2)
        self.assertEqual(4, res)

    def test_rechner_gibt_fehler_wenn_beide_args_nicht_zahlen(self): # neu
        self.assertRaises(ValueError, self.rech.add, 'zwei', 'drei')

if __name__ == '__main__':
    unittest.main()

```

Der neue Test überprüft, ob eine   <font color=#0000e6> ``ValueError``</font>  exception ausgelöst wurde. Zusätzlich wurde die Methode   <font color=#0000e6> ``setUp``</font>  verwendet, die zur Initialisierung der Tests verwendet werden kann. Das Test-Ergebnis sieht zunächst folgendermaßen aus, da noch kein   <font color=#0000e6> ``ValueError``</font>  im  eigentlichen code ausgelöst wird:
<pre>
> nosetests 
.F
======================================================================
FAIL: test_rechner_gibt_fehler_wenn_beide_args_nicht_zahlen (test.test_rechner.TddBeispiel)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/j/Johannes.Elmsheuser/cip_home/python/mytest/test/test_rechner.py", line 14, in test_rechner_gibt_fehler_wenn_beide_args_nicht_zahlen
    self.assertRaises(ValueError, self.rech.add, 'zwei', 'drei')
AssertionError: ValueError not raised
----------------------------------------------------------------------
Ran 2 tests in 0.015s
FAILED (failures=1)
</pre>

Der Code muss also folgendermassen verbessert werden:
```python
class Rechner(object):
 
    def add(self, x, y):
        number_typen = (int, float, complex)
 
        if isinstance(x, number_typen) and isinstance(y, number_typen): # neu
            return x + y
        else:
            raise ValueError
```

Um alle Kombinationsmöglichkeiten zu überprüfen, werden noch weitere Tests hinzugefügt:  
```python
import unittest
from app.rechner import Rechner

class TddBeispiel(unittest.TestCase):
  
    def setUp(self):
        self.rech = Rechner()

    def test_rechner_add_method_gibt_richtiges_ergebnis(self):
        res = self.rech.add(2,2)
        self.assertEqual(4, res)

    def test_rechner_gibt_fehler_wenn_beide_args_nicht_zahlen(self):
        self.assertRaises(ValueError, self.rech.add, 'zwei', 'drei')

    def test_rechner_gibt_fehler_wenn_x_arg_keine_zahl(self): # neu
        self.assertRaises(ValueError, self.rech.add, 'zwei', 3)
 
    def test_rechner_gibt_fehler_wenn_y_arg_keine_zahl(self): # neu
        self.assertRaises(ValueError, self.rech.add, 2, 'drei')

if __name__ == '__main__':
    unittest.main()
```

Alle 4 Tests funktionieren nun erfolgreich:
<pre>
> nosetests 
....
----------------------------------------------------------------------
Ran 4 tests in 0.020s
OK
</pre>